# Clone and install bwa
https://github.com/lh3/bwa

In [1]:
!cd /home/laura/projects && git clone https://github.com/lh3/bwa.git
!cd /home/laura/projects/bwa; make

Cloning into 'bwa'...
remote: Enumerating objects: 4398, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 4398 (delta 38), reused 53 (delta 26), pack-reused 4311
Receiving objects: 100% (4398/4398), 1.72 MiB | 0 bytes/s, done.
Resolving deltas: 100% (3123/3123), done.
gcc -c -g -Wall -Wno-unused-function -O2 -DHAVE_PTHREAD -DUSE_MALLOC_WRAPPERS   utils.c -o utils.o
gcc -c -g -Wall -Wno-unused-function -O2 -DHAVE_PTHREAD -DUSE_MALLOC_WRAPPERS   kthread.c -o kthread.o
gcc -c -g -Wall -Wno-unused-function -O2 -DHAVE_PTHREAD -DUSE_MALLOC_WRAPPERS   kstring.c -o kstring.o
gcc -c -g -Wall -Wno-unused-function -O2 -DHAVE_PTHREAD -DUSE_MALLOC_WRAPPERS   ksw.c -o ksw.o
gcc -c -g -Wall -Wno-unused-function -O2 -DHAVE_PTHREAD -DUSE_MALLOC_WRAPPERS   bwt.c -o bwt.o
gcc -c -g -Wall -Wno-unused-function -O2 -DHAVE_PTHREAD -DUSE_MALLOC_WRAPPERS   bntseq.c -o bntseq.o
gcc -c -g -Wall -Wno-unused-function -O2 -DHAVE_PTHREAD -DUSE_MALLOC_

In [1]:
bwa = "/home/laura/projects/bwa/bwa"

# Create bwa index

In [2]:
canine_macaque_fasta = "/home/laura/projects/virus-watch-data/canine_macaque_ref/combined_genomes.dna.toplevel.fa"

In [4]:
!$bwa index


Usage:   bwa index [options] <in.fasta>

Options: -a STR    BWT construction algorithm: bwtsw, is or rb2 [auto]
         -p STR    prefix of the index [same as fasta name]
         -b INT    block size for the bwtsw algorithm (effective with -a bwtsw) [10000000]
         -6        index files named as <in.fasta>.64.* instead of <in.fasta>.* 

         `-a div' do not work not for long genomes.



In [6]:
%%time
!$bwa index $canine_macaque_fasta

[bwa_index] Pack FASTA... 42.63 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=10736379650, availableWord=767451092
[BWTIncConstructFromPacked] 10 iterations done. 99999986 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 199999986 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 299999986 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 399999986 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 499999986 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 599999986 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 699999986 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 799999986 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 899999986 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 999999986 characters processed.
[BWTIncConstructFromPacked] 110 iterations done.

# Align seqs

In [3]:
import os
import glob
from tqdm import tqdm

TQDM_BAR_FORMAT = (
    "{l_bar}{bar}| {n_fmt}/{total_fmt} [elapsed: {elapsed} remaining: {remaining}]"
)

seqtk = "/home/laura/projects/seqtk/seqtk"

In [20]:
bp = "PRJNA665227"
fastq_folder = f"/home/laura/projects/virus-watch-data/{bp}/raw"
filenames = glob.glob(f"{fastq_folder}/*_2.fastq.gz")

bwa_outfolder = f"/home/laura/projects/virus-watch-data/{bp}/bwa"

In [21]:
len(filenames)

106

In [22]:
%%time
processed_srrs = [i.split("/")[-1].split(".sam.gz")[0] for i in glob.glob(f"{bwa_outfolder}/*.sam.gz")]

with tqdm(total=len(filenames), bar_format=TQDM_BAR_FORMAT) as pbar:
    pbar.set_description("Files processed")
    for file in filenames:
        # Check if file was already processed
        if file.split("/")[-1].split(".fastq.gz")[0] not in processed_srrs:
            print(file.split("/")[-1].split(".fastq.gz")[0])
            bwa_out = f"{bwa_outfolder}/{file.split('/')[-1].replace('.fastq.gz', '')}.sam.gz"
            sam_out = f"{bwa_outfolder}/{file.split('/')[-1].replace('.fastq.gz', '')}_unmapped.txt"
            new_file = f"{bwa_outfolder}/raw/{file.split('/')[-1].replace('.gz', '')}"

            # Align to reference
            !$bwa mem \
                -t 60 \
                $canine_macaque_fasta \
                $file | gzip -3 > $bwa_out

            # Get names of all unmapped reads
            !samtools view \
                --threads 60 \
                -f 4 \
                $bwa_out | cut -f 1 > $sam_out

            # Create new fastq file including only unmapped sequences
            !$seqtk subseq $file $sam_out > $new_file

            pbar.update(n=1)

Files processed:   0%|                                                                                                          | 0/106 [elapsed: 00:00 remaining: ?]

SRR12698521_2
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 6818182 sequences (600000016 bp)...
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3304.306 CPU sec, 66.939 real sec
[M::mem_process_seqs] Processed 6818182 reads in 3388.543 CPU sec, 58.461 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3284.585 CPU sec, 58.139 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3316.809 CPU sec, 56.775 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3343.842 CPU sec, 58.192 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3305.926 CPU sec, 56.675 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3337.210 CPU s

Files processed:   1%|▉                                                                                               | 1/106 [elapsed: 1:05:01 remaining: 113:47:54]

CPU times: user 1min 3s, sys: 12.7 s, total: 1min 16s
Wall time: 1h 5min 1s


In [ ]:
# %%time
# with tqdm(total=len(filenames), bar_format=TQDM_BAR_FORMAT) as pbar:
#     pbar.set_description("Files processed")
#     for file in filenames:
#         bwa_out = f"{bwa_outfolder}/{file.split('/')[-1].replace('.fastq.gz', '')}.sam.gz"
#         sam_out = f"{bwa_outfolder}/{file.split('/')[-1].replace('.fastq.gz', '')}_unmapped.txt"
#         new_file = f"{bwa_outfolder}/raw/{file.split('/')[-1].replace('.gz', '')}"

#         # Align to reference
#         !$bwa mem \
#             -t 60 \
#             $canine_macaque_fasta \
#             $file | gzip -3 > $bwa_out

#         # Get names of all unmapped reads
#         !samtools view \
#             --threads 60 \
#             -f 4 \
#             $bwa_out | cut -f 1 > $sam_out

#         # Create new fastq file including only unmapped sequences
#         !$seqtk subseq $file $sam_out > $new_file
        
#         pbar.update(n=1)

Files processed:   0%|                                                                                           | 0/105 [elapsed: 00:00 remaining: ?]

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 6818182 sequences (600000016 bp)...
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 2355.552 CPU sec, 49.705 real sec
[M::mem_process_seqs] Processed 6818182 reads in 2270.289 CPU sec, 39.975 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 2285.282 CPU sec, 39.459 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 2192.362 CPU sec, 37.842 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 2110.195 CPU sec, 36.639 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 2167.190 CPU sec, 37.500 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 2251.849 CPU sec, 38.919 rea

Files processed:   1%|▊                                                                                  | 1/105 [elapsed: 58:41 remaining: 101:43:43]

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 6818182 sequences (600000016 bp)...
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3647.032 CPU sec, 70.188 real sec
[M::mem_process_seqs] Processed 6818182 reads in 3644.887 CPU sec, 62.869 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3592.679 CPU sec, 61.570 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3647.446 CPU sec, 62.701 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3580.851 CPU sec, 61.170 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3684.398 CPU sec, 64.007 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3645.346 CPU sec, 62.570 rea

Files processed:   2%|█▌                                                                                | 2/105 [elapsed: 1:53:49 remaining: 97:09:46]

...

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 6818182 sequences (600000016 bp)...
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3127.344 CPU sec, 64.333 real sec
[M::mem_process_seqs] Processed 6818182 reads in 3150.757 CPU sec, 54.512 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3048.323 CPU sec, 52.333 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3055.152 CPU sec, 52.428 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3058.960 CPU sec, 52.521 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 2989.132 CPU sec, 51.271 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 2992.012 CPU sec, 51.328 rea

Files processed:   6%|████▋                                                                            | 6/105 [elapsed: 7:13:00 remaining: 138:08:50]

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 6818182 sequences (600000016 bp)...
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3574.709 CPU sec, 69.776 real sec
[M::mem_process_seqs] Processed 6818182 reads in 3691.500 CPU sec, 63.686 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3626.544 CPU sec, 61.039 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3644.158 CPU sec, 62.652 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3640.763 CPU sec, 61.028 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3561.110 CPU sec, 60.853 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3596.393 CPU sec, 60.166 rea

Files processed:   7%|█████▍                                                                           | 7/105 [elapsed: 8:09:44 remaining: 122:20:14]

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 6818182 sequences (600000016 bp)...
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3491.914 CPU sec, 66.672 real sec
[M::mem_process_seqs] Processed 6818182 reads in 3628.902 CPU sec, 62.491 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3491.842 CPU sec, 58.597 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3540.602 CPU sec, 61.109 real sec
[M::process] read 6818182 sequences (600000016 bp)...
[M::mem_process_seqs] Processed 6818182 reads in 3519.950 CPU sec, 59.084 real sec
[M::process] read 6818182 sequences (600000016 bp)...


Also remove reads from barcode files:

In [23]:
%%time
barcode_files = glob.glob(f"{fastq_folder}/*_1.fastq.gz")

with tqdm(total=len(filenames), bar_format=TQDM_BAR_FORMAT) as pbar:
    pbar.set_description("Barcode files processed")
    
    for bc_file in barcode_files:
        sam_out = f"{bwa_outfolder}/{bc_file.split('/')[-1].replace('.fastq.gz', '').replace('_1', '_2')}_unmapped.txt"
        new_bc_file = f"{bwa_outfolder}/raw/{bc_file.split('/')[-1].replace('.gz', '')}"

        # Create new fastq file including only unmapped sequences
        !$seqtk subseq $bc_file $sam_out > $new_bc_file
        
        pbar.update(n=1)

Barcode files processed: 100%|███████████████████████████████████████████████████████████████████████████| 106/106 [elapsed: 6:11:22 remaining: 00:00]

CPU times: user 6min 17s, sys: 1min 22s, total: 7min 39s
Wall time: 6h 11min 22s


# Percentage of reads aligned to host